This is our notebook for 6.853 project. Our goal is to reduce mode-collapse when training GANs. We aim to do this by using existing literature on multiplayer, zero-sum games to phrase the GAN problem as a polymatrix game. Then, we aim to use the reduction to the high-stakes lawyer game. 

In [1]:
import numpy

In [2]:
print('hi')

hi
